In [1]:
from miniscope_tiffs import crop_and_convert_miniscope, get_all_CropROIs, getVideosInFolder, GetMiniscopeDirs, combine_multi_folders
import numpy as np
import glob
import os
import cv2
import scipy.io
# import cv2 

C:\Users\gjb326\Image_Analysis\miniscope_tiffs.py:263: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if tiff_name is not 'test.tiff':


In [2]:
# fdir1 = 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/Experiment_test/18_25_10/MiniLFOV/'
# fdir2 = 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/Experiment_test/18_32_52/MiniLFOV/'
# fdirs = [fdir1, fdir2]

# combine_multi_folders(fdirs, tiff_name='msCam.tiff', avi_names = '',
#             cropROI=None, circleDef=[], spatialDownSample=2, temporalDownSample=2,
#             frame_filter=None, verbose=True, show_vid=False)
# # still need to combine timestamps and ori file

First we extract the directories with miniscope recordings and behav cam recordings.

Setup the parameters so it knows what to look for and return
Folder structures should look like the miniscope default, i.e.


[topdir]/[animal]/[2022_dayfolder]/[sessionfolder]/[MiniLFOV or BehavCam]/

You might need to change the had coded year and camera names based on your recordings


In [3]:
# import this
# animals = ['Hipp16942'] # ['Hipp16941', 'Hipp16942', 'Hipp16943']
# topdir = 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/'
animals = ['TestMouse1','TestMouse2']
topdir = 'C:/Users/gjb326/Desktop/RecordingData/AlejandroGrau/'
    
miniscope_tiff_name = 'msCam.tiff'
behavcam_tiff_name = 'behavCam.tiff'
HasTiff = None

allMiniFolders, allBehavFolders, parentMiniFolders, parentBehavFolders = \
    GetMiniscopeDirs(topdir, animals, TiffDirsOnly=HasTiff, miniscope_tiff_name=miniscope_tiff_name, behavcam_tiff_name=behavcam_tiff_name)

allMiniFolders
allMiniFolders = [allMiniFolders[0]]
allMiniFolders


['C:/Users/gjb326/Desktop/RecordingData/AlejandroGrau/TestMouse1/2022_07_05/17_09_41/MiniLFOV']

In [4]:
# animals = ['Hipp16942'] # ['Hipp16941', 'Hipp16942', 'Hipp16943']
# topdir = 'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/'
# HasTiff = None
# allMiniFolders, allBehavFolders, parentMiniFolders, parentBehavFolders = \
#     GetMiniscopeDirs(topdir, animals, TiffDirsOnly=None, miniscope_tiff_name='', behavcam_tiff_name='')

# parentMiniFolders

In [5]:
## Optional way to make a csv file list all of the session directories
makeDirFile = False
if makeDirFile:
    delimiter = '&&'
    import csv
    for animal in animals:
        x, xx, parentMiniFolders, yy = \
            GetMiniscopeDirs(topdir, [animal], TiffDirsOnly=None, miniscope_tiff_name='', behavcam_tiff_name='')
        parentMiniFolders
        fname = str(topdir + animal + '/' + animal + '_directory_list.csv')
        with open(fname, 'w') as f:
            # using csv.writer method from CSV package
            write = csv.writer(f)
            write.writerow([str('SessionDir' + delimiter)])
            for subdir in parentMiniFolders:
                write.writerow([str(subdir + delimiter)])

In [6]:
# miniscopeFolders = allMiniFolders
# check out pathlib library for / \ probs
## PARAMETERS
ksize = 3 # recommend a small filter, 3 pixels usually
kern  = np.ones([ksize,ksize]) # small median filter to remove salt and pepper
s_ds  = 2 # spatial  down sample
t_ds  = 2 # temporal down sample

video_name = '' # expression labelling miniscope avi files, such as 'msCam' (if 'msCam1.avi', 'msCam2.avi',...)
                # or '' (if '0.avi', '1.avi', etc). Starts at the lowest and goes to the highest

# use_prev_crop = True
miniscope_tiff_name = 'test.tiff'
tiff_name  = miniscope_tiff_name
crop_shape = 'prev' # 'rect' # 'rect' # 'circle', 'rectangle', or 'square'. For no cropping: crop_shape=[]
use_mask   = False # if cropping with a circle, can mask out the area outside of the circle
verbose    = False # more or less print details
# if you use circle but don't want the masking, just use square instead

# Get the crop ROIs for each folder, store in a dictionary 'crop_ROIs'
# mask_ROIs is a dict of circular masks if you want to use circle and mask out the periphery
if (crop_shape != None) and (crop_shape != 'prev'):
    cropROIs, maskROIs  = get_all_CropROIs(allMiniFolders, avi_names=video_name, shape=crop_shape, verbose=verbose)
    
for subFolder in allMiniFolders:
    if crop_shape == 'prev':
        prevCrop = str(subFolder + '/Crop_params.mat')
        try:
            temp = scipy.io.loadmat(prevCrop)
            crop = temp['cropROI']
            print('Using previous crop file:')
            print('     -  ' + prevCrop)
        except ValueError:
                print('Prev crop file not found!')
    elif crop_shape != None: 
        crop = cropROIs[subFolder]
    else:
        crop = None
    print('Crop:  ' + str(crop))
    
    if use_mask: mask=maskROIs[subFolder]
    else: mask=None
    # Now do the actual combining and cropping 
    crop_and_convert_miniscope(dataFolder=subFolder, tiff_name=tiff_name, avi_names=video_name,
            cropROI=crop, circleDef=mask, spatialDownSample=s_ds, temporalDownSample=t_ds,
            frame_filter=kern, verbose=verbose, multicore=True)

Using previous crop file:
     -  C:/Users/gjb326/Desktop/RecordingData/AlejandroGrau/TestMouse1/2022_07_05/17_09_41/MiniLFOV/Crop_params.mat
Crop:  [[135  33]
 [588 511]]
     ~~~ Combining  12 video in C:/Users/gjb326/Desktop/RecordingData/AlejandroGrau/TestMouse1/2022_07_05/17_09_41/MiniLFOV ~~~
     ~~~ MULTIPROCESSING ~~~
Number of CPU: 24
Video processing using 24 processes...
Time taken: 99.5204644203186
Parameters saved to:  C:/Users/gjb326/Desktop/RecordingData/AlejandroGrau/TestMouse1/2022_07_05/17_09_41/MiniLFOV\Crop_params22.mat
Done! Movie saved to:  C:/Users/gjb326/Desktop/RecordingData/AlejandroGrau/TestMouse1/2022_07_05/17_09_41/MiniLFOV/test.tiff


In [ ]:
# behavFolders = [r'C:\Users\gjb326\Desktop\RecordingData\GarrettBlair\APA_aquisition\Hipp16941\2022_06_10\17_09_18\BehavCam']
                   
# check out pathlib library for / \ probs
## PARAMETERS
ksize = 3 # recommend a small filter, 3 pixels usually
kern  = np.ones([ksize,ksize]) # small median filter to remove salt and pepper
s_ds  = 2 # spatial  down sample
t_ds  = 2 # temporal down sample

video_name = '' # expression labelling miniscope avi files, such as 'msCam' (if 'msCam1.avi', 'msCam2.avi',...)
                # or '' (if '0.avi', '1.avi', etc). Starts at the lowest and goes to the highest
tiff_name  = behavcam_tiff_name
crop_shape = None #'sq' # 'rect' # 'circle', 'rectangle', or 'square'. For no cropping: crop_shape=[]
use_mask   = False # if cropping with a circle, can mask out the area outside of the circle
verbose    = True # more or less print details
# if you use circle but don't want the masking, just use square instead

# Get the crop ROIs for each folder, store in a dictionary 'crop_ROIs'
# mask_ROIs is a dict of circular masks if you want to use circle and mask out the periphery
if crop_shape != None:
    cropROIs, maskROIs  = get_all_CropROIs(allBehavFolders, avi_names=video_name, shape=crop_shape, verbose=verbose)
    cropROIs

for subFolder in allBehavFolders:
    if crop_shape == 'prev': # os.path.isfile(prevCrop) and use_prev_crop:
        prevCrop = str(subFolder + '/Crop_params.mat')
        try:
            temp = scipy.io.loadmat(prevCrop)
            crop = temp['cropROI']
            print('Using previous crop file:')
            print('     -  ' + prevCrop)
        except ValueError:
                print('Prev crop file not found!')
    elif crop_shape != None: 
        crop = cropROIs[subFolder]
    else:
        crop = None
    print('Crop:  ' + str(crop))
    
    if use_mask: mask=maskROIs[subFolder]
    else: mask=None
    # Now do the actual combining and cropping
    crop_and_convert_miniscope(dataFolder=subFolder, tiff_name=tiff_name, avi_names=video_name,
            cropROI=crop, circleDef=mask, spatialDownSample=s_ds, temporalDownSample=t_ds,
            frame_filter=kern, verbose=verbose)

## ~ Fin ~


The rest is just a demo of how to playback imaging and behavior together

In [ ]:
from tifffile import imwrite, imread
import pandas as pd
from miniscope_tiffs import scroll_images
import numpy as np
import cv2
topdir = r'C:/Users/gjb326/Desktop/RecordingData/GarrettBlair/APA_aquisition/Hipp16942/2022_06_14/20_00_14/'
t1 = str(topdir + '/MiniLFOV/timestamps.dat')
f1 = str(topdir + '/MiniLFOV/msCam.tiff')
f2 = str(topdir + '/BehavCam/behavCam.tiff')


ms = imread(f1)
bh = imread(f2)
bg = np.mean(ms, axis=0) - 20

In [ ]:
t1 = str(topdir + '/MiniLFOV/timestamps.csv')
t2 = str(topdir + '/BehavCam/timestamps.csv')
ms_dat = pd.read_csv(t1)
ms_t = ms_dat['Time Stamp (ms)']
beh_dat = pd.read_csv(t2)
beh_t = beh_dat['Time Stamp (ms)']

numFrames, h, w = bh.shape
tds = np.round(len(beh_t)/numFrames).astype(int)
tds
beh_t = beh_t[::tds]

numFrames, h, w = ms.shape
tds = np.round(len(ms_t)/numFrames).astype(int)
tds
ms_t = ms_t[::tds]

beh_frames = np.round(np.interp(ms_t, beh_t, np.arange(0, bh.shape[0]))).astype('int');
# beh_frames = np.round(np.interp(beh_t, ms_t, np.arange(0, numFrames))).astype('int');
beh_frames

In [ ]:
# cv2.namedWindow("msFrame")
frate = 11
numFrames, h, w = ms.shape

msf = ms[0]
scale_percent = 50 # percent of original size
width = int(msf.shape[1] * scale_percent / 100)
height = int(msf.shape[0] * scale_percent / 100)
dim = (width, height)
  
im_all = None
msframe2show = np.arange(0, numFrames)

for i in msframe2show: # ms.shape[0]
    msf = ms[i]
    bhf = bh[beh_frames[i]]
#     print('mstime: ' + str(ms_t.iloc[i]) + '     bhtime: ' + str(beh_t.iloc[beh_frames[i]]))
    msdff = 4*(msf-bg)
    msdff[msdff<0] = 0
    msdff[msdff>255] = 255
    msdff = msdff.astype('uint8')
    
    msf = cv2.resize(msf, dim, interpolation = cv2.INTER_AREA)
    msdff = cv2.resize(msdff, dim, interpolation = cv2.INTER_AREA)
    bhf = cv2.resize(bhf, dim, interpolation = cv2.INTER_AREA)
    
    im = np.hstack((msf, msdff, bhf))

    if im_all is None:
        im_all  = np.empty([len(msframe2show), msf.shape[0], msf.shape[1]*3], dtype=np.uint8)
    im_all[i] = im
    
#     cv2.imshow('msFrame', im)
#     key = cv2.waitKey(np.round(1000/frate).astype('int'))
# #     key = cv2.waitKey(1)
#     if key == 27:# if ESC is pressed, exit loop
#         cv2.destroyAllWindows()
#         break
cv2.destroyAllWindows()

In [ ]:
scroll_images(im_all)